# Воспроизведение получения данных для исследования ЦПУР «Качество проведения оценки регулирующего воздействия в России: что показывает сплошной анализ текстовых данных?»

Этот ноутбук – единая точка запуска всех скриптов, которые __собирают и обрабатывают данные__ с regulation.gov.ru и sozd.duma.gov.ru для дальнейшего использования в анализе.

Ноутбук состоит из трех частей: __инициализации__, __получения датасета__, аналогичного датасету, [опубликованному на сайте ИНИД]('https://data-in.ru/data-catalog/datasets/177/'), и __нормализации__ заполнений граф сводных отчетов на отсутствующие, мусорные и содержательные.

Получение датасета реализовано двумя способами: полной перегенерацией датасета, воспроизводящей работу аналитиков ЦПУР, и скачиванием датасета с сайта ИНИД.

<div class="alert alert-block alert-info">
<b>Важно:</b> Сбор исходного датасета, размещённого на ИНИД, занимает <b>длительное время</b>: требуется обойти несколько тысяч страниц электронных порталов, а затем конвертировать, распознавать и парсить длинные документы. Поэтому рекомендуем пропустить эту часть и вместо неё переходить к части «Вариант 2: скачать датасеты с ИНИД»
</div>

## Оглавление:
* [Инициализация](#test)
* [Вариант 1: скачать датасеты с ИНИД](#data-in-download)
* [Вариант 2: полностью перегенерировать данные](#full-data-gen)
* [Нормализация](#norm)

## Установка зависимостей и инициализация <a class="anchor" id="test"></a>

In [6]:
# Установим стандартные библиотеки через pip
!pip install --quiet -r pip-requirements.txt

# Установим библиотеку, разработанную ЦПУР для анализа отчётов
!./install_report_parser.sh

Cloning into 'report_parser'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.(24/58)
remote: Compressing objects: 100% (48/48), done.
remote: Total 58 (delta 19), reused 34 (delta 6), pack-reused 0
Receiving objects: 100% (58/58), 390.61 KiB | 3.99 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [9]:
# Загрузим конфигурационный файл
import json

config = json.load(open('config.json', 'r'))
workdir = config['working_directory']

In [32]:
# Добавим в PYTHONPATH ссылки на вспомогательные скрипты

import os
import sys

relative_lib_paths = ['utils/report_parser/',
                      'utils/cleanup_utils/',
                      'utils',
                      'scraping',
                      'gathering',
                      'normalization'
                     ]

absolute_lib_paths = [os.path.abspath(x) for x in relative_lib_paths]
for path in absolute_lib_paths:
    sys.path.insert(0, path)

from utils.report_parser.report import Report
# Протестируем, что Report загрузился
report_example = 'utils/report_parser/examples/report.html'
Report(report_example)

from utils.db_helper import DBHelper
# Протестируем, что DBHelper загрузился
_ = DBHelper(config['database'])

In [11]:
# Создадим рабочую папку, если её ещё нет

if not os.path.isdir(workdir):
    os.mkdir(workdir)

In [12]:
# Загрузим другие полезные вещи

import pandas as pd
import pickle
import json

## Вариант 1: скачать датасеты с ИНИД <a class="anchor" id="data-in-download"></a>

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Заменить ссылку с Росгидромета на новый датасет после релиза
</div>

In [48]:
# Скачаем и сложим датасеты в workdir/datasets

import zipfile
import urllib.request

data_dir = os.path.join(workdir, 'datasets/')
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

url = '''
    https://ds1.data-in.ru/Rosgidromet/Zagryaznenie_poverkhnostnih_vod_RF_176/Zagryaznenie_poverkhnostnih_vod_RF_176_23.09.21.zip
'''
with urllib.request.urlopen(url) as response, open(archive, 'wb') as out_file:
    data = response.read() # a `bytes` object
    out_file.write(data)
    
with zipfile.ZipFile(archive, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

## Вариант 2: полностью перегенерировать данные <a class="anchor" id="full-data-gen"></a>

<div class="alert alert-block alert-info">
<b>Важно:</b> Если вы выполнили предыдущий пункт, этот проходить необзятельно.
</div>

[Перейти к следующей части](#norm)

### Скачаем данные проектов на ОРВ

#### Данные с общей таблицы всех проектов

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Нужно переименовать таблицы и колонки в базе, чтобы было как в датасете на ИНИД
</div>

In [1]:
!scraping/regulation_parser.py -c config.json

/Users/partofspeech/Code/Repos/cpur/projects/orv_reports
Начинается скачивание файла дампа.
Queue submitted.
column "regulation_project_id" of relation "project" does not exist
LINE 1: INSERT INTO project (regulation_project_id, views_num, comme...
                             ^

120826 current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

120825 current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

120824 current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

120823 current transaction is aborted, commands ignored until end of transaction block

120823 current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, comm

#### Файлы сводных отчетов и текстов НПА

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Убрать лишние аргументы, типы документов 
</div>

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Переименования
</div>

In [1]:
# тут надо переписать закачивание
!./scraping/download_files.py -c config.json -i sozd

Traceback (most recent call last):
  File "./scraping/download_files.py", line 252, in <module>
    tasks, failed = make_queue(input_type, doc_type, degree)
  File "./scraping/download_files.py", line 164, in make_queue
    result = select_from_database(input_type, document_type, impact_degree)
  File "./scraping/download_files.py", line 220, in select_from_database
    return connection.select_statement(statement)
  File "/Users/partofspeech/Code/Repos/cpur/projects/orv_reports/utils/db_helper.py", line 68, in select_statement
    c.execute(statement)
psycopg2.errors.UndefinedColumn: column "number" does not exist
LINE 1: SELECT number, document_link FROM duma WHERE NOT document_li...
               ^



### Скачаем данные законопроектов, внесенных в Госдуму

#### Сбор метаданных по АПИ

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Запуск скрепера по АПИ
</div>

In [ ]:
!./duma_parser.py -c config.json

#### Файлы законопроектов

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Запуск даунлоадера по регулейшен
</div>

In [ ]:
!./download_files.py -c config.json -i duma

### Соберём сводные отчёты из файлов

На этом этапе мы с помощью библиотеки [report-parser]('https://github.com/CAG-ru/report_parser') преобразуем содержимое файлов, которые пока хранятся в формате отдельных файлов, в коллекцию питоновских объектов типа `Report` – то есть объектов, содержащих тексты и таблицы в формате `pandas.DataFrame`.

Это долгий процесс, можете оставить скрипт работать, и пойти выпить самовар чаю – результаты на всякий случай сохранятся на диск.

In [13]:
files_df = pd.DataFrame(columns=['regulation_project_id',
                                'relative_path'])

for degree in ['Низкая', 'Высокая', 'Средняя']:
    files_dir = os.path.join(workdir, 
                            'main_ria_report') + '_' + degree   
    pickled_path = os.path.join(workdir, 
                            'preparsed_reports') + '_' + degree
    
    reports = []
    
    for fn in os.listdir(files_dir):
        report_path = os.path.join(files_dir, fn)
        project_id = int(os.path.splitext(fn)[0])
        files_df = files_df.append(pd.Series({
            'regulation_project_id': project_id,
            'relative_path' : report_path
            }), ignore_index=True)
        try:
            report= Report(report_path)
            report.parse()
            reports.append((project_id, report))
        except Exception as e:
            print(fn, e)
            
    with open(pickled_path, 'wb') as f:
        pickle.dump(reports, f)

15096.docx File is not a zip file
20367.pdf Расширение pdf не поддерживается


### Структурируем данные сводных отчетов

На этом этапе проведём парсинг объектов типа Report в плоские таблицы: 1 основную и 15 вспомогательных.
В основной таблице одной строчке соответствует сводный отчёт одного проекта, а колонки соответствуют графам отчёта, которые не предполагают множественных ответов.

В случаях, когда графа отчёта может быть заполнена целым списком значений (например, «Цели предполагаемого регулирования»), для неё создаётся отдельная таблица с `one-to-many` соответствием: например, одному и тому же отчёту могут соответствовать несколько строк с разными целями регулирования.

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Поменять названия переменных, таблиц и тп
</div>

In [23]:
# Заберем метаинформацию, она поможет нам с парсингом

db = DBHelper(config['database'])
query = '''
    SELECT id, degreeregulatoryimpact_title
    FROM public.project
'''
task = pd.DataFrame(db.select_statement(query),
                   columns=['regulation_project_id', 
                            'impact_degree'])


In [17]:
# Подцепим назад записанные на диск Report'ы

reports = []
for degree in ['Низкая', 'Высокая', 'Средняя']:
    pickled_dir = os.path.join(workdir, 
                               'preparsed_reports') + '_' + degree 
    reports.extend(pickle.load(open(pickled_dir, 'rb')))

In [18]:
# Загрузим пустые датафреймы, куда будем складывать результаты

with open("gathering/df_primers", "rb") as f:
    df_primers = pickle.load(f)
main_df = df_primers["main_df"]
otm_tables = df_primers["otm_tables"]

In [19]:
# Запустим парсеры, которые структурируют информацию из отчетов

from gather_to_dataframe_utils import add_report_to_df, fill_blanks

bad_reports = []
for project_id, report in reports:
    main_df = add_report_to_df(project_id, report, main_df, otm_tables, bad_reports)

In [20]:
# Удалим лишние колонки – они возникают, когда кто-то случайно создает лишний заголовок в отчете
main_df = main_df.iloc[:, :73]

# Непустой 'header: id' – хороший предиктор адекватности отчета, 
# т.к. он заполняется автоматически и должен быть всегда
main_df = main_df[~main_df['header: id'].isnull()]

# Добавим нужные колонки и удалим ненужные
main_df.drop(['header: id', 'header: regulation_project_id',
              'header: start', 'header: end'], 
             inplace=True, axis=1)
main_df = main_df.merge(task, on='regulation_project_id', how='inner')

# Пометим специальным значением non-applicable графы, которые и не должны были быть заполнены
main_df = main_df.apply(fill_blanks, axis=1)
# Степень регулирующего воздействия больше не нужна
main_df.drop(['impact_degree'], 
             inplace=True, axis=1)

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Стандартную форму не нужно искать по всем колонкам
</div>

In [21]:
def remove_standard_form(text):
    if pd.isnull(text) or type(text) == int:
        return text
    text = text.replace('(указываются полное и краткое наименования)', '')
    text = text.replace('(есть/нет)', '')
    text = text.replace('(место для текстового описания)', '')
    text = text.replace('(дней с момента принятия проекта нормативного правового акта)', '')
    return text

for column in main_df.columns:
    main_df[column] = main_df[column].apply(remove_standard_form)

In [25]:
parsed_dir = os.path.join(workdir, 'datasets')

if not os.path.isdir(parsed_dir):
    os.mkdir(parsed_dir)

main_df.to_csv(os.path.join(parsed_dir, 'ria_reports_main.csv'), 
             index=False)

for otm_name, otm_df in otm_tables.items():
    if otm_name == 'business':
        otm_df[0].to_csv(os.path.join(parsed_dir,
                'ria_reports_business_sizes_as_is.csv'), 
                index=False)
        otm_df[1].to_csv(os.path.join(parsed_dir,
                'ria_reports_business_profit_loss.csv'), 
                index=False)
        otm_df[1].to_csv(os.path.join(parsed_dir,
            'ria_reports_business_sizes_to_be.csv'), 
             index=False)
        continue
        
    otm_df.to_csv(os.path.join(parsed_dir, 'ria_reports_' \
                               + otm_name + '.csv'), 
                               index=False)

## Нормализация <a class="anchor" id="norm"></a>

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Описать, что здесь происходит
</div>

In [28]:
# Загрузим датасеты

data_dir = os.path.join(workdir, 'datasets/')
main_df = pd.read_csv(data_dir + 'ria_reports_main.csv')

otm_tables_fnames = sorted([fn for fn in os.listdir(data_dir) if not (fn.startswith('main') or fn.startswith('.'))])
otm_tables = {fn[:-4]: pd.read_csv(data_dir + fn) for fn in otm_tables_fnames}
otm_tables.keys()

dict_keys(['ria_reports_business_profit_loss', 'ria_reports_business_sizes_as_is', 'ria_reports_business_sizes_to_be', 'ria_reports_cancel_duties', 'ria_reports_expenses', 'ria_reports_goals', 'ria_reports_group_changes', 'ria_reports_group_expenses', 'ria_reports_groups', 'ria_reports_kpi', 'ria_reports_main', 'ria_reports_necessary_measures', 'ria_reports_new_functions', 'ria_reports_notification_info', 'ria_reports_public_discussion', 'ria_reports_risks'])

<div class="alert alert-block alert-warning">
<b>ToDo:</b> Вынести clean_up и remove_standard_form в отдельный модуль в utils
</div>

In [29]:
def clean_up(text):
    text = str(text)
    text = text.strip()
    text = text.lower()
    if is_junk(text):
        return '$#*!'
        
    text = text.rstrip('.')
    text = text.rstrip(';')
    text = text.replace('«', '"').replace('»', '"')
    return text

test = pd.Series([
    '', '\r', 'dfd\n', '\r\n', '\n\r\n', 
    'u', '.',
    '-', '- --', 
    '____', 
    'nan', 'нет',
    '(место для текстового описания)',
    '-(место для текстового описания)'
])
test.apply(clean_up)

NameError: name 'is_junk' is not defined

In [16]:
# Поиск самых частых значений
def most_freq(series, num):
    return sorted(Counter(series).items(), key=lambda item: (-item[1], item[0]))[:num]

# Поиск самых коротких заполнений
def shortest(series, num):
    return sorted(Counter(series).items(), key=lambda item: (len(item[0]), item[1]))[:num]

# Добываем топ самых коротких и частых значений
def get_column_features(df, column, top_freq):
    cleaned_field = df[column].apply(clean_up)
    mf = pd.DataFrame(most_freq(cleaned_field, top_freq))
    sh = pd.DataFrame(shortest(cleaned_field, top_freq))
    return mf, sh

### Разметка мусорных и отсутствующих заполнений

In [34]:
with open('normalization/junk/junk_by_field.json', 'r') as fp:
    junk_by_field = json.load(fp)
        
    
def fill_df_info(dataframe, df_name):
    """
    Функция принимает датафрейм, сравнивает значение нужных полей
    с мусорными, проставляет соответствующую оценку
    и возвращает новый датафрейм
    """
    if df_name not in all_fields.keys():
        print('No such table in dataset.')
        return
    
    if 'header: id' in dataframe.columns:
        id_column = 'header: id'
    else:
        id_column = 'id'
    
    result_df = pd.DataFrame()
    for index, row in dataframe.iterrows():
        result_df.loc[index, id_column] = row[id_column]
        
        for short, long in all_fields.get(df_name).items():    
            value = dataframe.loc[index, long]
            value = clean_up(value)

            # Проверяем, есть ли текущее вхождение 
            # в "мусоре" для этого поля goals@timing
            junk_set = junk_by_field.get(short)
            if junk_set is not None and value in junk_set:
                valid = 0
            else:        
                valid = 1

            result_df.loc[index, short] = value
            result_df.loc[index, str(short + '_valid')] = valid
    return result_df

junk_main_df = fill_df_info(main_df, 'main')
junk_main_df.to_excel('junk/junk_main_df.xlsx', header=True)

for _ in otm_tables.keys():
    if _ in all_fields.keys():
        df = otm_tables.get(_)
        result = fill_df_info(df, _)
        result.to_excel(f'junk/junk_{_}_df.xlsx', header=True)

NameError: name 'all_fields' is not defined